## NER - Named Entity Recognition


### Module imports


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

# set random seeds to make this notebook easier to replicate
tf.keras.utils.set_random_seed(33)

### Exploring the data


We will be using a dataset from Kaggle.

The original data consists of four columns:

- the sentence number,
- the word,
- the part of speech of the word
- and the tags.

A few tags are:

- geo: geographical entity
- org: organization
- per: person
- gpe: geopolitical entity
- tim: time indicator
- art: artifact
- eve: event
- nat: natural phenomenon
- O: filler word

The prepositions in the tags mean:

- I: Token is inside an entity.
- B: Token begins an entity.

**Example:**

**"Sharon flew to Miami on Friday"**

The tags would look like:

```
Sharon B-per
flew   O
to     O
Miami  B-geo
on     O
Friday B-tim
```

**"Sharon Floyd flew to Miami on Friday"**

```
Sharon B-per
Floyd  I-per
flew   O
to     O
Miami  B-geo
on     O
Friday B-tim
```


In [2]:
# Original data

data = pd.read_csv("data/ner_dataset.csv", encoding="ISO-8859-1")
data.head(25)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


We are using a preprocessed version of the data which was generated from the original data


In [3]:
# Exploring the preprocessed data (loading from small group)

train_sents = open('data/small/train/sentences.txt', 'r').readline()
train_labels = open('data/small/train/labels.txt', 'r').readline()

print('SENTENCE:', train_sents)
print('SENTENCE LABEL:', train_labels)

print('ORIGINAL DATA:\n', data.head())

del (data, train_sents, train_labels)

SENTENCE: Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

SENTENCE LABEL: O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O

ORIGINAL DATA:
     Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O


Loading the data for training validation and testing


In [4]:
def load_data(file_path):
    with open(file_path, 'r', encoding="ISO-8859-1") as file:
        data = np.array([line.strip() for line in file.readlines()])
    return data

Data is divided into sentences and labels for each set


In [5]:
# Loading the data data from large group

train_sentences = load_data('data/large/train/sentences.txt')
train_labels = load_data('data/large/train/labels.txt')

val_sentences = load_data('data/large/val/sentences.txt')
val_labels = load_data('data/large/val/labels.txt')

test_sentences = load_data('data/large/test/sentences.txt')
test_labels = load_data('data/large/test/labels.txt')

In [6]:
print(len(train_sentences))
print(len(val_sentences))
print(len(test_sentences))

33570
7194
7194


In [7]:
print(train_sentences[:5])
print(train_labels[:5])

['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'
 'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "'
 'They marched from the Houses of Parliament to a rally in Hyde Park .'
 'Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .'
 "The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton ."]
['O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O'
 'O O O O O O O O O O O O O O O O O O B-per O O O O O O O O O O O'
 'O O O O O O O O O O O B-geo I-geo O' 'O O O O O O O O O O O O O O O'
 'O O O O O O O O O O O B-geo O O B-org I-org O O O B-gpe O O O B-geo O']


In [8]:
print(val_sentences[:5])
print(val_labels[:5])

["Russia 's victory put the eight-time Olympic champions into the quarterfinals and also clinched a spot for Sweden ."
 'Slovakia advanced with a win over the United States ( 02-Jan ) on Saturday , leaving one remaining spot from Group-B .'
 'China has announced its sixth human bird flu death .'
 'Chinese health officials said Wednesday a 35-year-old woman from Sichuan province died last week from the H5N1 strain .'
 'Meanwhile , Turkish officials say bird flu killed an 11-year-old girl on her way to a hospital Wednesday .']
['B-geo O O O O O O O O O O O O O O O O B-org O'
 'B-geo O O O O O O B-geo I-geo O O O O B-tim O O O O O O B-art O'
 'B-org O O O O O O O O O'
 'B-gpe O O O B-tim O O O O B-geo O O O O O O B-nat B-tim O'
 'O O B-gpe O O O O O O O O O O O O O O B-tim O']


In [9]:
print(test_sentences[:5])
print(test_labels[:5])

['Argentina benefits from rich natural resources , a highly literate population , an export-oriented agricultural sector , and a diversified industrial base .'
 "Although one of the world 's wealthiest countries 100 years ago , Argentina suffered during most of the 20th century from recurring economic crises , persistent fiscal and current account deficits , high inflation , mounting external debt , and capital flight ."
 "A severe depression , growing public and external indebtedness , and a bank run culminated in 2001 in the most serious economic , social , and political crisis in the country 's turbulent history ."
 "Interim President Adolfo RODRIGUEZ SAA declared a default - the largest in history - on the government 's foreign debt in December of that year , and abruptly resigned only a few days after taking office ."
 "His successor , Eduardo DUHALDE , announced an end to the peso 's decade-long 1-to-1 peg to the US dollar in early 2002 ."]
['B-geo O O O O O O O O O O O O O O O O

### Encoding the data


#### Encoding the sentences


We will use `tf.keras.layers.TextVectorization`. We will explicitly pass `standardize = None`. By default, `standardize = 'lower_and_strip_punctuation'`.

This means the parser will remove all punctuation and make everything lowercase. This may influence the NER task, since an upper case in the middle of a sentence may indicate an entity. The sentences in the dataset are already split into tokens, and all tokens, including punctuation, are separated by a whitespace. The punctuations are also labeled. So everything will just be split into single tokens and then mapped to a positive integer.

`tf.keras.layers.TextVectorization` will also pad the sentences. But padding won't impact at all the model's output.

- padding token: "", integer mapped: 0
- unknown token "[UNK]", integer mapped: 1


We will use the object `tf.keras.layers.TextVectorization` and the appropriate parameters to build a function that inputs an array of sentences and outputs an adapted sentence vectorizer and its vocabulary list.


In [10]:
def get_sentence_vectorizer(sentences):

    # Setting the random seed
    tf.keras.utils.set_random_seed(33)

    """
    Parameters:
    sentences (list of str): Sentences for vocabulary adaptation.

    Returns:
    sentence_vectorizer (tf.keras.layers.TextVectorization): TextVectorization layer for sentence tokenization.
    vocab (list of str): Extracted vocabulary.
    """

    # Define TextVectorization object with the appropriate standardize parameter
    sentence_vectorizer = tf.keras.layers.TextVectorization(standardize=None)

    # Adapt the sentence vectorization object to the given sentences
    sentence_vectorizer.adapt(sentences)

    # Get the vocabulary
    vocab = sentence_vectorizer.get_vocabulary()

    return sentence_vectorizer, vocab

In [11]:
# Using the function to get the adapted vectorizer and vocabulary on a subset of the training data

test_vectorizer, test_vocab = get_sentence_vectorizer(train_sentences[:1000])

print(f"Test vocab size: {len(test_vocab)}")



Test vocab size: 4650


In [12]:
print(test_vocab[:20])

['', '[UNK]', 'the', '.', ',', 'in', 'of', 'to', 'a', 'and', 'The', "'s", 'for', 'is', 'has', 'said', 'on', 'have', 'that', 'from']


In [13]:
# Testing the test vectorizer

sentence = "I like learning new NLP models !"

sentence_vectorized = test_vectorizer(sentence)

print(f"Sentence: {sentence}\nSentence vectorized: {sentence_vectorized}")

Sentence: I like learning new NLP models !
Sentence vectorized: [ 296  314    1   59    1    1 4649]


In [14]:
# Getting the sentence vectorizer adapted on the complete training data

sentence_vectorizer, vocab = get_sentence_vectorizer(train_sentences)

In [15]:
print(f"Vocab size: {len(vocab)}")

Vocab size: 29847


In [16]:
# Testing the complete vectorizer on the same example

sentence = "I like learning new NLP models !"

sentence_vectorized = sentence_vectorizer(sentence)

print(f"Sentence: {sentence}\nSentence vectorized: {sentence_vectorized}")

Sentence: I like learning new NLP models !
Sentence vectorized: [  654  1211  6896    69     1 11044  4126]


#### Encoding the labels


The process is a bit simpler than encoding the sentences, because there are only a few tags. Also, there will be one extra tag to represent the padded token that some sentences may have included. Padding will not interfere at all in this task.

There is no meaning in having an UNK token for labels and also the padding token will be another number different from 0 for the labels. So, TextVectorization is not a good choice here.

We will code our own label vectorizer.


In [17]:
print(f"Sentence: {train_sentences[0]}")
print(f"Labels: {train_labels[0]}")

Sentence: Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Labels: O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O


The following function extract all the different tags in a given set of labels (all the training example set) and return us the uniqe ones.


In [18]:
def get_tags(labels):
    tag_set = set()  # Define an empty set
    for el in labels:
        for tag in el.split(" "):
            tag_set.add(tag)
    tag_list = list(tag_set)
    tag_list.sort()
    return tag_list

In [19]:
tags = get_tags(train_labels)
print(len(tags))
print(tags)

17
['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']


The following function generate **tag map** and **tag_map_reverse**.
- A mapping between the tags and positive integers
- A reverse mapping between the positive integers and tags

In [20]:
def make_tag_map(tags):
    tag_map = {}
    tag_map_rev = {}
    for i, tag in enumerate(tags):
        tag_map[tag] = i
        tag_map_rev[i] = tag
    return tag_map, tag_map_rev

In [21]:
tag_map, tag_map_rev = make_tag_map(tags)

print(tag_map)
print(tag_map_rev)

{'B-art': 0, 'B-eve': 1, 'B-geo': 2, 'B-gpe': 3, 'B-nat': 4, 'B-org': 5, 'B-per': 6, 'B-tim': 7, 'I-art': 8, 'I-eve': 9, 'I-geo': 10, 'I-gpe': 11, 'I-nat': 12, 'I-org': 13, 'I-per': 14, 'I-tim': 15, 'O': 16}
{0: 'B-art', 1: 'B-eve', 2: 'B-geo', 3: 'B-gpe', 4: 'B-nat', 5: 'B-org', 6: 'B-per', 7: 'B-tim', 8: 'I-art', 9: 'I-eve', 10: 'I-geo', 11: 'I-gpe', 12: 'I-nat', 13: 'I-org', 14: 'I-per', 15: 'I-tim', 16: 'O'}


#### Padding the labels


TextVectorization has already padded the sentences, so we must ensure that the labels are properly padded as well.Tensorflow has built-in functions for padding `tf.keras.utils.pad_sequences`.

We will pad the vectorized labels with the value -1. We will not use 0 to simplify loss masking and evaluation in further steps.

This is because to properly classify one token, a log softmax transformation will be performed and the index with greater value will be the index label. Since index starts at 0, it is better to keep the label 0 as a valid index, even though it is possible to also use 0 as a mask value for labels, but it would require some tweaks in the model architecture or in the loss computation.


#### Label vectorizer


We will build the label vectorizer.

This function inputs a list of labels and a tag mapping and outputs their respective label ids via a tag map lookup.


In [22]:
def label_vectorizer(labels, tag_map, DEBUG_PRINT=False):
    """
    Convert list of label strings to padded label IDs using a tag mapping.

    Parameters:
    labels (list of str): List of label strings.
    tag_map (dict): Dictionary mapping tags to IDs.
    Returns:
    label_ids (numpy.ndarray): Padded array of label IDs.
    """
    label_ids = []  # It can't be a numpy array yet, since each sentence has a different size

    # Each element in labels is a string of tags so for each of them:
    for element in labels:
        # Split it into single tokens. You may use .split function for strings. Be aware to split it by a blank space!
        tokens = element.split()

        # DEBUG PRINT
        if DEBUG_PRINT == True:
            print("tokens", tokens, "\n")

        # Use the dictionaty tag_map passed as an argument to the label_vectorizer function
        # to make the correspondence between tags and numbers.
        element_ids = []

        for token in tokens:
            # Tag map lookup
            # Appending the ids corresponding to the tokens in the label
            element_ids.append(tag_map[token])

        # DEBUG PRINT
        if DEBUG_PRINT == True:
            print("element_ids", element_ids, "\n")

        # Append the found ids to corresponding to the current element to label_ids list
        label_ids.append(element_ids)

    # DEBUG PRINT
    if DEBUG_PRINT == True:
        print("label_ids", label_ids, "\n")

    # Pad the elements
    label_ids = tf.keras.utils.pad_sequences(
        label_ids, padding='post', value=-1)

    # DEBUG PRINT
    if DEBUG_PRINT == True:
        print("label_ids", label_ids, "\n")

    return label_ids

Looking at a single example


In [23]:
print(f"Sentence: {train_sentences[2]}")
print(f"Labels: {train_labels[2]}")

Sentence: They marched from the Houses of Parliament to a rally in Hyde Park .
Labels: O O O O O O O O O O O B-geo I-geo O


In [24]:
print(f"Vectorized labels: {label_vectorizer([train_labels[2]], tag_map)}")

Vectorized labels: [[16 16 16 16 16 16 16 16 16 16 16  2 10 16]]


Looking at two examples together


In [25]:
print(f"Sentence: {train_sentences[2:4]}")

Sentence: ['They marched from the Houses of Parliament to a rally in Hyde Park .'
 'Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .']


In [26]:
print(f"Labels: {train_labels[2:4]}")

Labels: ['O O O O O O O O O O O B-geo I-geo O' 'O O O O O O O O O O O O O O O']


In [27]:
print(f"Vectorized labels: {label_vectorizer(train_labels[2:4], tag_map)}")

Vectorized labels: [[16 16 16 16 16 16 16 16 16 16 16  2 10 16 -1]
 [16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]]


### Building the dataset


We will build the dataset for training, validation and testing.

We will be using `tf.data.Dataset` class, which provides an optimized way to handle data to feed into a tensorflow model. It avoids keeping all the data in memory, thus it makes the training faster.

We will be using the `tf.data.Dataset.from_tensor_slices` function that converts any iterable into a Tensorflow dataset.

We can pass a tuple of `(sentences,labels)` and Tensorflow will understand that each sentence is mapped to its respective label, therefore it is expected that if a tuple of arrays is passed, both arrays have the same length.


In [28]:
def generate_dataset(sentences, labels, sentence_vectorizer, tag_map):
    sentences_ids = sentence_vectorizer(sentences)
    labels_ids = label_vectorizer(labels, tag_map=tag_map)
    dataset = tf.data.Dataset.from_tensor_slices((sentences_ids, labels_ids))
    return dataset

In [29]:
train_dataset = generate_dataset(
    train_sentences, train_labels, sentence_vectorizer, tag_map)
val_dataset = generate_dataset(
    val_sentences, val_labels,  sentence_vectorizer, tag_map)
test_dataset = generate_dataset(
    test_sentences, test_labels,  sentence_vectorizer, tag_map)

In [30]:
# Exploring information about the training data
# The number of vocabulary tokens (including <PAD>)

g_vocab_size = len(vocab)

print(f"Num of vocabulary words in the training set: {g_vocab_size}")

print()

print('The training size is', len(train_dataset))
print('The validation size is', len(val_dataset))

print()

print('An example of the first sentence is\n\t',
      next(iter(train_dataset))[0].numpy())
print('An example of its corresponding label is\n\t',
      next(iter(train_dataset))[1].numpy())

Num of vocabulary words in the training set: 29847

The training size is 33570
The validation size is 7194

An example of the first sentence is
	 [1046    6 1121   18 1832  232  543    7  528    2  158    5   60    9
  648    2  922    6  192   87   22   16   54    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
An example of its corresponding label is
	 [16 16 16 16 16 16  2 16 16 16 16 16  2 16 16 16 16 16  3 16 16 16 16 16
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

#### About RNNs and LSTMs inputs

Tensorflow implementation of RNNs (in particular LSTMs) allow us to pass a variable size of input sentences. However this cannot be done in the same batch. You must assure that, for each batch, the shapes for our input tensors are the same.

For this purpose, the size of the padding should not influence the final result. Therefore, it does not matter if we perform the padding for each batch or in the entire dataset.


### Model building


Inputs will be sentences represented as tensors that are fed to a model with,

- An Embedding layer,
- A LSTM layer
- A Dense layer
- A log softmax layer

We may choose between outputting only the very last LSTM output for each sentence, but we may also request the LSTM to output every value for a sentence - this is what we want.

We will need every output, because the idea is to label every token in the sentence and not to predict the next token or even make an overall classification task for that sentence.

This implies that when we input a single sentence, such as `[452, 3400, 123, 0, 0, 0]`, the expected output should be an array for each word ID, with a length equal to the number of tags. This output is obtained by applying the LogSoftfmax function for each of the `len(tags)` values.

So, in the case of the example array with a shape of `(6,)`, the output should be an array with a shape of `(6, len(tags))`.

In our case, we've seen that each sentence in the training set is 104 values long, so in a batch of, say, 64 tensors, the model shoud input a tensor of shape `(64,104)` and output another tensor with shape `(64,104,17)`.


#### About tensorflow layers


**`tf.keras.Sequential`**

- This combinator applies layers serially (by function composition). It is a tensorflow model object.

---

**`tf.keras.layers.Embedding`**

- Initializes the embedding layer `Embedding(input_dim, output_dim, mask_zero = False)`
- `input_dim` is the expected range of integers for each tensor in the batch. Note that the `input_dim` is not related to array size, but to the possible range of integers expected in the input. Usually this is the vocabulary size, but it may differ by 1, depending on further parameters.
- `output_dim` is the number of elements in the word embedding (some choices for a word embedding size range from 150 to 300, for example). Each word processed will be assigned an array of size `output_dim`. So if one array of shape (3,) is passed (example of such an array `[100,203,204]`), then the Embedding layer should have output shape (3,output_dim).
- `mask_zero` is a boolean telling whether 0 is a mask value or not. If `mask_zero = True`, then some considerations must be done:
  - The value 0 should be reserved as the mask value, as it will be ignored in training.
  - We need to add 1 in `input_dim`, since now Tensorflow will consider that one extra 0 value may show up in each sentence.

---

**`tf.keras.layers.LSTM`**

- `LSTM(units, return_sequences)` Builds an LSTM layer with hidden state and cell sizes equal to `units`.
- `units`: It is the number of `LSTM` cells we will create to pass every input to. In this case, set the `units` as the Embedding `output_dim`. This is just a choice, in fact there is no static rule preventing one from choosing any amount of LSTM units.
- `return_sequences`: A boolean, telling whether we want to return every output value from the LSTM cells. If `return_sequences = False`, then the LSTM output shape will be `(batch_size, units)`. Otherwise, it is `(batch_size, sentence_length, units)`, since there will be an output for each word in the sentence.

---

**`tf.keras.layers.Dense`**

- `Dense(units, activation)`
- `units`: It is the number of units chosen for this dense layer, i.e., it is the dimensionality of the output space. In this case, each value passed through the Dense layer must be mapped into a vector with length `num_of_classes` (in this case, `len(tags)`).
- `activation`: This is the activation that will be performed after computing the values in the Dense layer. Since the Dense layer comes before the LogSoftmax step, we can pass the LogSoftmax function as activation function here. \*\*We can find the implementation for LogSoftmax under `tf.nn`. So we may call it as `tf.nn.log_softmax`.


In [31]:
def NER(len_tags, vocab_size, embedding_dim=50):
    """
    Create a Named Entity Recognition (NER) model.

    Parameters:
    len_tags (int): Number of NER tags (output classes).
    vocab_size (int): Vocabulary size.
    embedding_dim (int, optional): Dimension of embedding and LSTM layers (default is 50).

    Returns:
    model (Sequential): NER model.
    """

    model = tf.keras.Sequential(name='sequential')

    # Add the tf.keras.layers.Embedding layer. Do not forget to mask out the zeros!
    model.add(tf.keras.layers.Embedding(input_dim=vocab_size +
              1, output_dim=embedding_dim, mask_zero=True))

    # Add the LSTM layer. Make sure we are passing the right dimension (defined in the docstring above)
    # and returning every output for the tf.keras.layers.LSTM layer and not the very last one.
    model.add(tf.keras.layers.LSTM(units=embedding_dim, return_sequences=True))

    # Add the final tf.keras.layers.Dense with the appropriate activation function. Remember we must pass the activation function itself and not its call!
    # We must use tf.nn.log_softmax instead of tf.nn.log_softmax().
    model.add(tf.keras.layers.Dense(
        units=len_tags, activation=tf.nn.log_softmax))

    return model

### Masked loss and metrics


**Custom Accuracy Function**

Before training the model, we need to create our own function to compute the accuracy. Tensorflow has built-in accuracy metrics but we cannot pass values to be ignored. This will impact the calculations, since we must remove the padded values.

Usually, the metric that inputs true labels and predicted labels and outputs how many times the predicted and true labels match is called `accuracy`.

In some cases, however, there is one more step before getting the predicted labels. This may happen if, instead of passing the predicted labels, a vector of probabilities is passed.

In such case, there is a need to perform an `argmax` for each prediction to find the appropriate predicted label. Such situations happen very often, therefore Tensorflow has a set of functions, with prefix `Sparse`, that performs this operation in the backend.

Unfortunately, it does not provide values to ignore in the accuracy case. This is what we will work on now.

Note that the model's prediction has 3 axes:

- the number of examples (batch size)
- the number of words in each example (padded to be as long as the longest sentence in the batch)
- the number of possible targets (the 17 named entity tags).


**Custom Loss Function**

Another important function is the loss function. In this case, we will use the Cross Entropy loss, but we need a multiclass implementation of it, also we may look for its `Sparse` version.

Tensorflow has a SparseCategoricalCrossentropy loss function, which is already imported by the name SparseCategoricalCrossEntropy.

The arguments we will need:

- `from_logits`: This indicates if the values are raw values or normalized values (probabilities). Since the last layer of the model finishes with a LogSoftMax call, the results are **not** normalized - they do not lie between 0 and 1.
- `ignore_class`: This indicates which class should be ignored when computing the crossentropy. Remember that the class related to padding value is set to be 0.


#### Custom masked loss function

In [32]:
def masked_loss(y_true, y_pred):
    """
    Calculate the masked sparse categorical cross-entropy loss.

    Parameters:
    y_true (tensor): True labels.
    y_pred (tensor): Predicted logits.

    Returns:
    loss (tensor): Calculated loss.
    """

    # Calculate the loss for each item in the batch. Remember to pass the right arguments, as discussed above!
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, ignore_class=-1)

    # Use the previous defined function to compute the loss
    loss = loss_fn(y_true, y_pred)

    return loss

In [33]:
# Trying out the custom loss function

true_labels = tf.constant([0, 1, 2, 0])

predicted_logits = tf.constant([[0.1, 0.6, 0.3],
                                [0.2, 0.7, 0.1],
                                [0.1, 0.5, 0.4],
                                [0.4, 0.4, 0.2]])

print(masked_loss(true_labels, predicted_logits))

tf.Tensor(1.0508584, shape=(), dtype=float32)


#### Custom masked accuracy function

We will make a masked version of the accuracy function.

We will need to perform an argmax to get the predicted label for each element in the batch. We need to make sure to provide the appropriate axis in the argmax function.

Furthermore, remember to use only tensorflow operations.

Even though numpy has every function we will need, to pass it as a loss function and/or metric function, we must use tensorflow operations, due to internal optimizations that Tensorflow performs for reliable fitting.

The following tensorflow functions are already loaded in memory, so we can directly call them.

- `tf.equal`, equivalent to `np.equal`
- `tf.cast`, equivalent to `np.astype`
- `tf.reduce_sum`, equiavalent to `np.sum`
- `tf.math.argmax`, equivalent to `np.argmax`
- We may need `tf.float32` while casting


In [34]:
def masked_accuracy(y_true, y_pred):
    """
    Calculate masked accuracy for predicted labels.

    Parameters:
    y_true (tensor): True labels.
    y_pred (tensor): Predicted logits.

    Returns:
    accuracy (tensor): Masked accuracy.

    """

    # Calculate the loss for each item in the batch.
    # We must always cast the tensors to the same type in order to use them in training.
    # Since we will make divisions, it is safe to use tf.float32 data type.
    y_true = tf.cast(y_true, tf.float32)

    # Create the mask, i.e., the values that will be ignored
    mask = tf.not_equal(y_true, -1)
    mask = tf.cast(mask, tf.float32)

    # Perform argmax to get the predicted values
    y_pred_class = tf.argmax(y_pred, axis=-1)
    y_pred_class = tf.cast(y_pred_class, tf.float32)

    # Compare the true values with the predicted ones
    matches_true_pred = tf.equal(y_true, y_pred_class)
    matches_true_pred = tf.cast(matches_true_pred, tf.float32)

    # Multiply the acc tensor with the masks
    matches_true_pred *= mask

    # Compute masked accuracy
    # quotient between the total matches and the total valid values, i.e., the amount of non-masked values
    masked_acc = tf.reduce_sum(matches_true_pred) / \
        tf.maximum(tf.reduce_sum(mask), 1)

    return masked_acc

In [35]:
true_labels = [0, 1, 2, 0]

predicted_logits = [[0.1, 0.6, 0.3],
                    [0.2, 0.7, 0.1],
                    [0.1, 0.5, 0.4],
                    [0.4, 0.4, 0.2]]

print(masked_accuracy(true_labels, predicted_logits))

tf.Tensor(0.5, shape=(), dtype=float32)


### The model

In [36]:
model = NER(len(tag_map), len(vocab))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1492400   
                                                                 
 lstm (LSTM)                 (None, None, 50)          20200     
                                                                 
 dense (Dense)               (None, None, 17)          867       
                                                                 
Total params: 1513467 (5.77 MB)
Trainable params: 1513467 (5.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1492400   
                                                                 
 lstm (LSTM)                 (None, None, 50)          20200     
                                                                 
 dense (Dense)               (None, None, 17)          867       
                                                                 
Total params: 1513467 (5.77 MB)
Trainable params: 1513467 (5.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Note on padding

We will check now how padding does not affect the model's output. 

Of course the output dimension will change. If ten zeros are added at the end of the tensor, then the resulting output dimension will have 10 more elements (more specifically, 10 more arrays of length 17 each). 

However, those are removed from any calculation further on, so it won't impact at all the model's performance and training.

In [39]:
# Expanding dims is needed to pass it to the model, 
# since it expects batches and not single prediction arrays

x = tf.expand_dims(np.array([545, 467, 896]), axis = 0)
    
x_padded = tf.expand_dims(np.array([545, 467, 896, 0, 0, 0]), axis = 0)

In [40]:
pred_x = model(x)
pred_x_padded = model(x_padded)

print(f'x shape: {pred_x.shape}\nx_padded shape: {pred_x_padded.shape}')

x shape: (1, 3, 17)
x_padded shape: (1, 6, 17)


If the last three elements of `pred_x_padded` are removed, both `pred_x` and `pred_x_padded[:3]` must have the same elements.

In [41]:
np.allclose(pred_x, pred_x[:3])

True

Now one last check: let's see that both `pred_x` and `pred_x_padded` return the same loss and accuracy values. 

For that, we will need a `y_true` and `y_true_padded` arrays.

In [42]:
y_true = tf.expand_dims([16, 6, 12], axis = 0)

# Remember we mapped the padded values to -1 in the labels
y_true_padded = tf.expand_dims([16,6,12,-1,-1,-1], axis = 0) 

print(f"masked_loss is the same: {np.allclose(masked_loss(y_true,pred_x), masked_loss(y_true_padded,pred_x_padded))}")

print(f"masked_accuracy is the same: {np.allclose(masked_accuracy(y_true,pred_x), masked_accuracy(y_true_padded,pred_x_padded))}")

masked_loss is the same: True
masked_accuracy is the same: True


### Model compilation

We will compile the model as follows:

- Use the Adam optimizer to compute the stochastic gradient descent, with learning rate 0.01
- Use the loss function `masked_loss` as loss function,
- As evaluation metrics, we will use both masked_loss and masked_accuracy

In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=masked_loss,
              metrics=[masked_accuracy])

### Model training

We will train the model with `shuffle = True`, over 2 epochs and passing the validation dataset as `validation_data`.

We will run into an error if we just pass the datasets as they are right now, because they are not prepared in batches. We must use the method `.batch` that returns a dataset already divided in batches.

The fitting takes about 1 minute to run. Only the first epoch is slow, the following ones are much faster.

#### Epoch, Batch size = 2, 64

In [44]:
# Setting again a random seed to ensure reproducibility
tf.keras.utils.set_random_seed(33)

BATCH_SIZE = 64

model.fit(train_dataset.batch(BATCH_SIZE),
          validation_data = val_dataset.batch(BATCH_SIZE),
          shuffle=True,
          epochs = 2)

Epoch 1/2

525/525 [==============================] - 79s 138ms/step - loss: 0.2687 - masked_accuracy: 0.9303 - val_loss: 0.1397 - val_masked_accuracy: 0.9585
Epoch 2/2
525/525 [==============================] - 77s 147ms/step - loss: 0.1099 - masked_accuracy: 0.9657 - val_loss: 0.1361 - val_masked_accuracy: 0.9587


Model slightly overfitting

### Compute accuracy on test data

In [45]:
# Convert the sentences into ids
test_sentences_id = sentence_vectorizer(test_sentences)

# Convert the labels into token ids
test_labels_id = label_vectorizer(test_labels,tag_map)

print(test_labels_id, "\n")

# Rename to prettify next function call
y_true = test_labels_id


y_pred = model.predict(test_sentences_id)

[[ 2 16 16 ... -1 -1 -1]
 [16 16 16 ... -1 -1 -1]
 [16 16 16 ... -1 -1 -1]
 ...
 [ 3 16 16 ... -1 -1 -1]
 [16 16 16 ... -1 -1 -1]
 [16 16 16 ... -1 -1 -1]] 

225/225 [==============================] - 7s 19ms/step


In [46]:
print(f"The model's accuracy in test set is: {masked_accuracy(y_true,y_pred).numpy():.4f}")

The model's accuracy in test set is: 0.9585


#### Testing with single sentence string

In [47]:
test_sentences_id = sentence_vectorizer(test_sentences[0])
print(test_sentences_id.shape)
print(test_sentences_id)

(23,)
tf.Tensor(
[ 1633  2112    22  2968   778  1300     4     8  2102     1   964     4
    28 21100  1572   694     4     9     8  9656  2019   556     3], shape=(23,), dtype=int64)


In [48]:
y_pred = model.predict(test_sentences_id)

print(y_pred)

TypeError: in user code:

    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\U

So, we need to expand dimension before prediction

In [49]:
test_sentences_id = sentence_vectorizer(test_sentences[0])
print(test_sentences_id.shape)
print(test_sentences_id)

test_sentences_id = tf.expand_dims(test_sentences_id, axis=0)
print(test_sentences_id.shape)
print(test_sentences_id)

(23,)
tf.Tensor(
[ 1633  2112    22  2968   778  1300     4     8  2102     1   964     4
    28 21100  1572   694     4     9     8  9656  2019   556     3], shape=(23,), dtype=int64)
(1, 23)
tf.Tensor(
[[ 1633  2112    22  2968   778  1300     4     8  2102     1   964     4
     28 21100  1572   694     4     9     8  9656  2019   556     3]], shape=(1, 23), dtype=int64)


In [50]:
y_pred = model.predict(test_sentences_id)

print(y_pred)

1/1 [==============================] - 2s 2s/step
[[[-6.13779688e+00 -6.79265404e+00 -3.94498527e-01 -2.49608350e+00
   -7.98183680e+00 -1.91077375e+00 -4.73243332e+00 -4.50187969e+00
   -6.63349342e+00 -7.98887300e+00 -3.25325513e+00 -6.39309454e+00
   -8.42420769e+00 -5.00940895e+00 -6.00375175e+00 -6.55422211e+00
   -3.94616866e+00]
  [-8.59139919e+00 -8.20378208e+00 -7.79352379e+00 -9.53281593e+00
   -7.19473076e+00 -8.50618076e+00 -6.51852703e+00 -4.43424845e+00
   -6.59590054e+00 -7.35481358e+00 -7.44420767e+00 -8.73978138e+00
   -6.56122112e+00 -6.78192854e+00 -5.43081665e+00 -5.42684507e+00
   -2.97499523e-02]
  [-1.32201519e+01 -1.31385546e+01 -9.45972919e+00 -1.25458879e+01
   -1.26733685e+01 -1.05976772e+01 -1.40467691e+01 -4.20205116e+00
   -1.19679422e+01 -1.45431309e+01 -1.28799782e+01 -1.55161581e+01
   -1.30081711e+01 -1.34321537e+01 -1.34954643e+01 -7.04925346e+00
   -1.60904229e-02]
  [-1.20176935e+01 -1.20625668e+01 -1.00448771e+01 -1.63060589e+01
   -1.08135271e+01 

In [51]:
outputs = np.argmax(y_pred, axis=-1)
print(outputs)

[[ 2 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]]


In [52]:
outputs = outputs[0]
print(outputs)

[ 2 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]


#### Testing with list of sentence

In [53]:
test_sentences_id = sentence_vectorizer(test_sentences[0:2])
print(test_sentences_id.shape)
print(test_sentences_id)

print("="*70)

test_sentences_id = tf.expand_dims(test_sentences_id, axis=0)
print(test_sentences_id.shape)
print(test_sentences_id)

(2, 43)
tf.Tensor(
[[ 1633  2112    22  2968   778  1300     4     8  2102     1   964     4
     28 21100  1572   694     4     9     8  9656  2019   556     3     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [ 2086    65     6     2   146    11  9084   137   704   112   350     4
   1633  1223    96   217     6     2  2866   880    22 18793   173  8530
      4  6843  1767     9   824  2195  5217     4   353  1838     4  5145
   3090   939     4     9   148  2280     3]], shape=(2, 43), dtype=int64)
(1, 2, 43)
tf.Tensor(
[[[ 1633  2112    22  2968   778  1300     4     8  2102     1   964
       4    28 21100  1572   694     4     9     8  9656  2019   556
       3     0     0     0     0     0     0     0     0     0     0
       0     0     0     0     0     0     0     0     0     0]
  [ 2086    65     6     2   146    11  9084   137   704   112   350
       4  1633  1223    96   217     6     2  2866   88

In [54]:
y_pred = model.predict(test_sentences_id)

print(y_pred)

TypeError: in user code:

    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\U

In the case of multiple sentences, the output of `test_sentences_id = sentence_vectorizer(test_sentences[0:2])` itself becomes 2D.

So, we need not to expand the dimension further.

In [55]:
test_sentences_id = sentence_vectorizer(test_sentences[0:2])
print(test_sentences_id.shape)
print(test_sentences_id)

(2, 43)
tf.Tensor(
[[ 1633  2112    22  2968   778  1300     4     8  2102     1   964     4
     28 21100  1572   694     4     9     8  9656  2019   556     3     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [ 2086    65     6     2   146    11  9084   137   704   112   350     4
   1633  1223    96   217     6     2  2866   880    22 18793   173  8530
      4  6843  1767     9   824  2195  5217     4   353  1838     4  5145
   3090   939     4     9   148  2280     3]], shape=(2, 43), dtype=int64)


In [56]:
y_pred = model.predict(test_sentences_id)

print(y_pred)

1/1 [==============================] - 0s 123ms/step
[[[-6.1377969e+00 -6.7926545e+00 -3.9449853e-01 ... -6.0037522e+00
   -6.5542226e+00 -3.9461689e+00]
  [-8.5913982e+00 -8.2037811e+00 -7.7935233e+00 ... -5.4308162e+00
   -5.4268446e+00 -2.9749952e-02]
  [-1.3220152e+01 -1.3138554e+01 -9.4597282e+00 ... -1.3495462e+01
   -7.0492525e+00 -1.6090538e-02]
  ...
  [-1.4516864e+01 -1.4309671e+01 -1.3225335e+01 ... -1.4843252e+01
   -1.3103371e+01 -2.5033638e-05]
  [-1.4516864e+01 -1.4309671e+01 -1.3225335e+01 ... -1.4843252e+01
   -1.3103371e+01 -2.5033638e-05]
  [-1.4516863e+01 -1.4309671e+01 -1.3225335e+01 ... -1.4843252e+01
   -1.3103372e+01 -2.5033638e-05]]

 [[-1.1578687e+01 -1.1515079e+01 -1.1490126e+01 ... -1.0707664e+01
   -8.5873919e+00 -6.9320016e-04]
  [-1.0447474e+01 -1.0448552e+01 -9.0334644e+00 ... -1.3734333e+01
   -6.8000216e+00 -4.0983647e-02]
  [-1.1205212e+01 -9.6078587e+00 -1.0202290e+01 ... -1.4752798e+01
   -4.3839302e+00 -1.4190835e-02]
  ...
  [-1.1922279e+01 -1.180

In [57]:
outputs = np.argmax(y_pred, axis=-1)
print(outputs)

[[ 2 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16
  16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]
 [16 16 16 16 16 16 16 16 16 16 16 16  2 16 16 16 16 16  7 15 15 15 16 16
  16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16]]


#### Getting labels from label_ids

In [58]:
outputs[0]

array([ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16], dtype=int64)

In [59]:
# Tags for the first output
# Converting numpy object into list as they are not iterable

for i in list(outputs[0]):
    print(i, tag_map_rev[i])

2 B-geo
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O
16 O


### Predict function

In [60]:
def predict(sentence, model, sentence_vectorizer, tag_map):
    """
    Predict NER labels for a given sentence using a trained model.

    Parameters:
    sentence (str): Input sentence.
    model (tf.keras.Model): Trained NER model.
    sentence_vectorizer (tf.keras.layers.TextVectorization): Sentence vectorization layer.
    tag_map (dict): Dictionary mapping tag IDs to labels.

    Returns:
    predictions (list): Predicted NER labels for the sentence.

    """

    # Convert the sentence into ids
    sentence_vectorized = sentence_vectorizer([sentence])
    
    # Expand its dimension to make it appropriate to pass to the model
    # As we are passing the single sentence inside a list itself `sentence_vectorizer([sentence])`,
    # we need not to perform the `expand_dims` operation 
    # sentence_vectorized = tf.expand_dims(sentence_vectorized, axis=0)

    # Get the model output
    output = model.predict(sentence_vectorized)
    
    # Get the predicted labels for each token, using argmax function and specifying the correct axis to perform the argmax
    outputs = np.argmax(output, axis=-1)
    
    # Next line is just to adjust outputs dimension. Since this function expects only one input to get a prediction, outputs will be something like [[1,2,3]]
    # so to avoid heavy notation below, let's transform it into [1,2,3]
    outputs = outputs[0]

    pred = []

    # Iterating over every predicted token in outputs list
    for tag_idx in outputs:
        pred_label = tag_map_rev[tag_idx]
        pred.append(pred_label)
      
    return pred

In [61]:
sentence = "Many French citizens are goin to visit Morocco for summer"

predictions = predict(sentence, model, sentence_vectorizer, tag_map)

print(predictions, "\n")

for x,y in zip(sentence.split(' '), predictions):
    if y != 'O':
        print(x,y)

1/1 [==============================] - 0s 42ms/step
['O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'B-tim'] 

French B-gpe
Morocco B-geo
summer B-tim


In [62]:
sentence = "Sharon Floyd flew to Miami last Friday"

predictions = predict(sentence, model, sentence_vectorizer, tag_map)

print(predictions, "\n")

for x,y in zip(sentence.split(' '), predictions):
    if y != 'O':
        print(x,y)

1/1 [==============================] - 0s 50ms/step
['B-per', 'I-per', 'O', 'O', 'B-geo', 'O', 'B-tim'] 

Sharon B-per
Floyd I-per
Miami B-geo
Friday B-tim


In [63]:
# New york times news:

sentence = "Peter Parker , the White House director of trade and manufacturing policy of U.S , said in an interview on Sunday morning that the White House was working to prepare for the possibility of a second wave of the coronavirus in the fall , though he said it wouldn ’t necessarily come"

predictions = predict(sentence, model, sentence_vectorizer, tag_map)

print(predictions, "\n")

for x,y in zip(sentence.split(' '), predictions):
    if y != 'O':
        print(x,y)

1/1 [==============================] - 0s 63ms/step
['B-per', 'I-per', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] 

Peter B-per
Parker I-per
White B-org
House I-org
U.S B-org
Sunday B-tim
morning I-tim
White B-org
House I-org


### Save the trained model and load it again to test

#### Save the model

In [64]:
model.save('./model/ner_trained.keras')

In [65]:
model.save('./model/ner_trained.h5')

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


`.keras` is for the current versions of keras.

For legacy support we can use `.h5`

#### Load and use the model

In [66]:
loaded_model = tf.keras.models.load_model('./model/ner_trained.keras')

TypeError: Error when deserializing class 'Dense' using config={'name': 'dense', 'trainable': True, 'dtype': 'float32', 'units': 17, 'activation': {'module': 'builtins', 'class_name': 'function', 'config': 'log_softmax_v2', 'registered_name': 'function'}, 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: Unknown activation function '{'module': 'builtins', 'class_name': 'function', 'config': 'log_softmax_v2', 'registered_name': 'function'}' cannot be deserialized.